In [73]:
# 📌 **1. Import Library**
import requests
import json
import os
import base64
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# 📌 **2. Baca Data dari CSV**
csv_path = "data/anxiety_attack_dataset.csv"
data = pd.read_csv(csv_path)

# 📌 **3. Ambil 10 Data Secara Acak untuk Uji**
sample_data = data.sample(n=10, random_state=42)  # Pilih 10 baris acak untuk diuji

In [74]:
# 📌 **4. Transformasi Sesuai dengan TFX Pipeline**

# Daftar fitur pada dataset yang perlu di-encode
FEATURES = [
    "Age",
    "Sleep Hours",
    "Physical Activity (hrs/week)",
    "Caffeine Intake (mg/day)",
    "Alcohol Consumption (drinks/week)",
    "Stress Level (1-10)",
    "Heart Rate (bpm during attack)",
    "Breathing Rate (breaths/min)",
    "Sweating Level (1-5)",
    "Therapy Sessions (per month)",
    "Diet Quality (1-10)",
    "Gender",
    "Occupation",
    "Smoking",
    "Family History of Anxiety",
    "Dizziness",
    "Medication",
    "Recent Major Life Event"
]

# Label key
LABEL_KEY = "Anxiety Category Encoded"

In [75]:
def transformed_name(key):
    """Menambahkan suffix '_xf' untuk fitur yang telah ditransformasikan."""
    return f"{key}_xf"

In [76]:
# Melakukan Label Encoding pada fitur kategorikal
label_encoder_dict = {}
for feature in FEATURES:
    if sample_data[feature].dtype == 'object':  # jika fitur bertipe object (kategorikal)
        label_encoder = LabelEncoder()
        sample_data[feature] = label_encoder.fit_transform(sample_data[feature])
        label_encoder_dict[feature] = label_encoder  # Menyimpan encoder jika diperlukan

In [77]:
# **3. Memilih Fitur untuk Testing**
# Mengambil fitur yang sudah ter-encode dengan nama yang sesuai (_xf)
encoded_features = {transformed_name(feature): sample_data[feature] for feature in FEATURES}

In [78]:
# Menyiapkan data untuk request ke API
data_for_inference = pd.DataFrame(encoded_features).iloc[0].to_dict()

In [79]:
data_for_inference

{'Age_xf': 58.0,
 'Sleep Hours_xf': 3.3,
 'Physical Activity (hrs/week)_xf': 8.5,
 'Caffeine Intake (mg/day)_xf': 47.0,
 'Alcohol Consumption (drinks/week)_xf': 10.0,
 'Stress Level (1-10)_xf': 1.0,
 'Heart Rate (bpm during attack)_xf': 120.0,
 'Breathing Rate (breaths/min)_xf': 19.0,
 'Sweating Level (1-5)_xf': 3.0,
 'Therapy Sessions (per month)_xf': 4.0,
 'Diet Quality (1-10)_xf': 7.0,
 'Gender_xf': 0.0,
 'Occupation_xf': 3.0,
 'Smoking_xf': 0.0,
 'Family History of Anxiety_xf': 0.0,
 'Dizziness_xf': 0.0,
 'Medication_xf': 0.0,
 'Recent Major Life Event_xf': 0.0}

In [80]:
# **4. Kirim Request ke API**
api_url = "https://ml-pipeline-anxiety-attack-production.up.railway.app/v1/models/anxiety-model:predict"
json_data = prepare_json(data_for_inference)

response = requests.post(api_url, data=json_data)

In [81]:
# **5. Menampilkan Hasil Inference**
if response.status_code == 200:
    prediction = response.json().get("predictions")
    if prediction:
        prediction_value = prediction[0][0]
        
        # Mapping nilai prediksi ke label yang sesuai
        if prediction_value == 0:
            result = "Mild Anxiety"
        elif prediction_value == 1:
            result = "Moderate Anxiety"
        elif prediction_value == 2:
            result = "Panic Level Anxiety"
        elif prediction_value == 3:
            result = "Severe Anxiety"
        else:
            result = "Error: Unrecognized prediction value."
    else:
        result = "Error: No predictions found in response."
else:
    result = f"Error: {response.status_code}"

print(result)

Error: 400
